In [1]:
from lr.models.transformers.processor import filter_df_by_label
from lr.models.transformers.BertWrapper import BertWrapper
import numpy as np
import pandas as pd
from time import time
import shutil
import os

### Params

In [2]:
folder = "toy"

hyperparams = {"local_rank": -1,
               "max_seq_length": 200,
               "overwrite_cache": False,
               "num_train_epochs":1.0,
               "per_gpu_train_batch_size":32,
               "per_gpu_eval_batch_size":32,
               "gradient_accumulation_steps": 1,
               "learning_rate":5e-5,
               "weight_decay":0.0,
               "adam_epsilon": 1e-8,
               "max_grad_norm": 1.0,
               "max_steps": 7,
               "warmup_steps": 0,
               "save_steps": 6,
               "no_cuda":False,
               "n_gpu":1,
               "model_name_or_path":"bert",
               "output_dir":"bert",
               "random_state": 42,
               "fp16":False,
               "fp16_opt_level":"01",
               "device":"cpu",
               "verbose":False,
               "model_type": "bert",
               "pad_on_left":False,
               "pad_token":0,
               "pad_token_segment_id":0,
               "mask_padding_with_zero":True,
               "eval_sample_size":100,
               "n_cores":8,
               "base_path": "data/{}/cached_".format(folder)} 

## loading base model

In [3]:
my_bert = BertWrapper(hyperparams)

## Loading DFs

In [4]:
train_path = "data/{}/train.csv".format(folder)
test_path = "data/{}/dev.csv".format(folder)

df = pd.read_csv(train_path)
test = pd.read_csv(test_path)
test = test.sample(100, random_state=hyperparams["random_state"])

### Eval 1

In [5]:
pred = my_bert.predict(test, transform=True, mode="test")
lmap = my_bert.processor.get_label_map()
filtered = filter_df_by_label(test.dropna()).reset_index(drop=True)
before_acc = np.mean(filtered.label.map(lambda x: lmap[x]) == pred)

### Fit

In [6]:
global_step, tr_loss, train_time = my_bert.fit(df)

### Eval 2

In [7]:
pred = my_bert.predict(None, transform=False,path="data/toy/cached_test_200")
lmap = my_bert.processor.get_label_map()
filtered = filter_df_by_label(test.dropna()).reset_index(drop=True)
after_acc = np.mean(filtered.label.map(lambda x: lmap[x]) == pred)

### Results

In [8]:
assert tr_loss == 1.1454728543758392
assert before_acc == 0.31
assert after_acc == 0.4

In [9]:
if os.path.exists("example.log"):
    os.remove("example.log")

if os.path.exists("bert"):
    shutil.rmtree("bert")

    
paths = ["cached_dev_to_eval_200", "cached_test_200", "cached_train_200",
         "cached_train_to_eval_200"]

for path in paths:
    path = "data/{}/{}".format(folder, path)
    if os.path.exists(path):
        os.remove(path)
